# Just-in-time Compilation with [Numba](http://numba.pydata.org/) 

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import numba

## Using `numba.jit`

Numba offers `jit` which can used to decorate Python functions.

In [ ]:
def is_prime(n):
    if n <= 1:
        raise ArithmeticError('"%s" <= 1' % n)
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False
            
    return True

In [ ]:
n = np.random.randint(2, 10000000, dtype=np.int64) # Get a random integer between 2 and 10000000
print(n, is_prime(n))

In [ ]:
#is_prime(1)

In [ ]:
@numba.jit(forceobj=True)
def is_prime_jitted(n):
    if n <= 1:
        raise ArithmeticError('"%s" <= 1' % n)
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False

    return True

In [ ]:
numbers = np.random.randint(2, 100000, dtype=np.int64, size=10000)
%time p1 = [is_prime(n) for n in numbers]
%time p2 = [is_prime_jitted(n) for n in numbers]

## Using ` @numba.jit(nopython=True)` is equivalent to using ` @numba.njit`

In [ ]:
@numba.njit
def is_prime_njitted(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False

    return True

In [ ]:
numbers = np.random.randint(2, 100000, dtype=np.int64, size=1000)
%time p = [is_prime_njitted(n) for n in numbers]
%time p = [is_prime_njitted(n) for n in numbers]

## Use `cache=True` to cache the compiled function

In [ ]:
import math
from numba import njit

@njit(cache=True)
def is_prime_njitted_cached(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False

    return True

In [ ]:
numbers = np.random.randint(2, 100000, dtype=np.int64, size=1000)
%time p = [is_prime_njitted_cached(n) for n in numbers]
%time p = [is_prime_njitted_cached(n) for n in numbers]

## Eager compilation using function signatures

In [ ]:
import math
from numba import njit

@njit(['boolean(int64)', 'boolean(int32)'])
def is_prime_njitted_eager(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False

    return True

In [ ]:
numbers = np.random.randint(2, 1000000, dtype=np.int64, size=1000)
%time p1 = [is_prime_njitted_eager(n) for n in numbers]
%time p2 = [is_prime_njitted_eager(n) for n in numbers]

In [ ]:
p1 = [is_prime_njitted_eager(n) for n in numbers.astype(np.int32)]
#p2 = [is_prime_njitted_eager(n) for n in numbers.astype(np.float64)]

# Using `numba.jit` to speedup the computation of the Euclidean distance matrix 

In this notebook we implement a function to compute the Euclidean distance matrix using Numba's *just-in-time* compilation decorator. We compare it with the NumPy function we wrote before.

We will use two Numba functions here: The decorator ` @numba.jit` and `numba.prange`.

In [ ]:
import numpy as np
import numba

In [ ]:
@numba.jit(nopython=True)
def euclidean_numba1(x, y):
    """Euclidean square distance matrix using pure loops
    and no NumPy operations
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            r = 0.0
            for k in numba.prange(num_feat):
                r += (x[i][k] - y[j][k])**2
            dist_matrix[i][j] = r

    return dist_matrix

@numba.jit(nopython=True)
def euclidean_numba2(x, y):
    """Euclidean square distance matrix using loops
    and the `numpy.dot` operation
    """
    num_samples, num_feat = x.shape
    dist_matrix = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in numba.prange(num_samples):
            dist_matrix[i][j] = ((x[i] - y[j])**2).sum()

    return dist_matrix

Let's include here our numpy implementation for comparison.

In [ ]:
def euclidean_numpy(x, y):
    """Euclidean square distance matrix using numpy"""
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[:, np.newaxis].T
    xy = np.dot(x, y.T)
    return np.abs(x2 + y2 - 2. * xy)

### Note
Observe that we do the inner loop, which is a reduction, with `numba.prange`. `numba.prange` automatically takes care of data privatization and reductions.

### Exercise 1
Before runing the different functions, could you say which of the two numba implementations would be faster?

In [ ]:
# Let's check that they all give the same result
a = 10. * np.random.random([100, 10])

print(np.abs(euclidean_numpy(a, a) - euclidean_numba1(a, a)).max())
print(np.abs(euclidean_numpy(a, a) - euclidean_numba2(a, a)).max())

Our Numba implementations can be faster than the NumPy one for a list of small vectors. However, with larger vectors, the NumPy implementation is faster:

In [ ]:
nsamples = 100
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

In [ ]:
nsamples = 100
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)
%timeit euclidean_numba2(x, x)

In a more realistic case, our NumPy implementation is much faster:

In [ ]:
nsamples = 5000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_numpy(x, x)
%timeit euclidean_numba1(x, x)

## Calculating and plotting the [Mandelbrot set](https://en.wikipedia.org/wiki/Mandelbrot_set)

In [ ]:
X, Y = np.meshgrid(np.linspace(-2.0, 1, 1000), np.linspace(-1.0, 1.0, 1000))

def mandelbrot(X, Y, itermax):
    mandel = np.empty(shape=X.shape, dtype=np.int32)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            it = 0
            cx = X[i, j]
            cy = Y[i, j]
            x = 0.0
            y = 0.0
            while x * x + y * y < 4.0 and it < itermax:
                x, y = x * x - y * y + cx, 2.0 * x * y + cy
                it += 1
            mandel[i, j] = it
            
    return mandel

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)

%time m = mandelbrot(X, Y, 100)
    
ax.imshow(np.log(1 + m), extent=[-2.0, 1, -1.0, 1.0]);
ax.set_aspect('equal')
ax.set_ylabel('Im[c]')
ax.set_xlabel('Re[c]');

In [ ]:
@numba.njit(parallel=True)
def mandelbrot_jitted(X, Y, radius2, itermax):
    mandel = np.empty(shape=X.shape, dtype=np.int32)
    for i in numba.prange(X.shape[0]):
        for j in range(X.shape[1]):
            it = 0
            cx = X[i, j]
            cy = Y[i, j]
            x = cx
            y = cy
            while x * x + y * y < 4.0 and it < itermax:
                x, y = x * x - y * y + cx, 2.0 * x * y + cy
                it += 1
            mandel[i, j] = it
            
    return mandel

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)


%time m = mandelbrot_jitted(X, Y, 4.0, 100)
    
ax.imshow(np.log(1 + m), extent=[-2.0, 1, -1.0, 1.0]);
ax.set_aspect('equal')
ax.set_ylabel('Im[c]')
ax.set_xlabel('Re[c]');

### Getting parallelization information

In [ ]:
mandelbrot_jitted.parallel_diagnostics(level=3)

## Creating `ufuncs` using `numba.vectorize`

In [ ]:
from math import sin
from numba import float64, int64

def my_numpy_sin(a, b):
    return np.sin(a) + np.sin(b)

@np.vectorize
def my_sin(a, b):
    return sin(a) + sin(b)

@numba.vectorize([float64(float64, float64), int64(int64, int64)], target='parallel')
def my_sin_numba(a, b):
    return np.sin(a) + np.sin(b)

In [ ]:
x = np.random.randint(0, 100, size=9000000)
y = np.random.randint(0, 100, size=9000000)

%time _ = my_numpy_sin(x, y)
%time _ = my_sin(x, y)
%time _ = my_sin_numba(x, y)

### Vectorize the testing of prime numbers 

In [ ]:
@numba.vectorize('boolean(int64)')
def is_prime_v(n):
    if n <= 1:
        raise ArithmeticError(f'"0" <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False
            
    return True

In [ ]:
numbers = np.random.randint(2, 10000000000, dtype=np.int64, size=100000)
%time p = is_prime_v(numbers)

### Parallelize the vectorized function

In [ ]:
@numba.vectorize(['boolean(int64)', 'boolean(int32)'],
                 target='parallel')
def is_prime_vp(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False
            
    return True

In [ ]:
numbers = np.random.randint(2, 10000000000, dtype=np.int64, size=100000)
%time p1 = is_prime_v(numbers)
%time p2 = is_prime_vp(numbers)

In [ ]:
# Print the largest primes from to 1 and 10 millions
numbers = np.arange(1000000, 10000001, dtype=np.int32)
%time p1 = is_prime_vp(numbers)
primes = numbers[p1]

for n in primes[-10:]:
    print(n)

# Creating Generalized Ufuncs with Numba

Numba offers the `guvectorize` to generate **generalized ufuncs** which work of input arrays with different dimensions.

In [ ]:
import numpy as np
import numba

## Adding a constant to a vector

In [ ]:
@numba.guvectorize(['(f8[:], f8[:], f8[:])'], '(m),()->(m)')
def vec_add_const(x, y, z):
    for i in range(x.shape[0]):
        z[i] = x[i] + y[0]

In [ ]:
x = np.arange(10.0)
z = vec_add_const(x, 2.0)
print(x, z, sep='\n')

In [ ]:
x = np.arange(10.0).reshape(2, 5)
z = vec_add_const(x, 2.0)
print(x, z, sep='\n')

In [ ]:
x = np.arange(10.0).reshape(2, 5)
y = np.array([1., 2.])
z = vec_add_const(x, y)
print(x, z, sep='\n')

## Matrix Vector Multiplication

In [ ]:
@numba.guvectorize(['(f8[:, :], f8[:], f8[:])'], '(m,n),(n)->(m)')
def mat_vec_mult(x, y, z):
    for i in range(x.shape[0]):
        d = 0.0
        for j in range(x.shape[1]):
            d += x[i, j] * y[j]
        z[i] = d

In [ ]:
A = np.arange(9.0).reshape(3, 3)
x = np.array([1., 2., 3.])
z = mat_vec_mult(A, x)
print(A, z, sep='\n\n')

In [ ]:
A = np.arange(27.0).reshape(3, 3, 3)
x = np.array([1., 2., 3])
z = mat_vec_mult(A, x)
print(A, z, sep='\n\n')

## Matrix-Matrix Multiplication

In [ ]:
@numba.guvectorize(['(f8[:, :], f8[:, :], f8[:, :])'], '(m,n),(n,k)->(m, k)')
def mat_mul(x, y, z):
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            d = 0.0
            for k in range(x.shape[1]):
                d += x[i, k] * y[k, j]
            z[i, j] = d

In [ ]:
A = np.arange(9.0).reshape(3, 3)
B = np.arange(9.0, 24.0).reshape(3, 5)
C = mat_mul(A, B)
C_numpy = A @ B
print(C, C_numpy, sep='\n\n')
numba.guvectorize?